<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220118_sm_know_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
warnings.filterwarnings(action='ignore')

In [81]:
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]
train3 = train3[train3['idx'] != 25823]
train3 = train3[train3['idx'] != 18746]
train3 = train3[train3['idx'] != 19183]
train3 = train3[train3['idx'] != 21879]
train3 = train3[train3['idx'] != 21931]
train3 = train3[train3['idx'] != 26330]

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')


test2.loc[3982, test2.columns[1:]] = 0
test3.loc[3694, test3.columns[1:]] = 0
test3.loc[7090, test3.columns[1:]] = 0

In [82]:
know_train = [train1, train2, train3, train4]
know_test = [test1, test2, test3, test4]

In [83]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [84]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [85]:
years = ['2017', '2018', '2019', '2020']

rf_predicts = []
year_encoder = {}
int_col = {}
text_col = {}

for year, train, test in zip(years, know_train, know_test):
    print(year)
    encoders = {}
    int_temp = []
    text_temp = []
    for col in train.columns:
        if col == 'ID':
            continue
        try:
            train[col] = train[col].map(int)
            int_temp.append(col)
        except:
            encoder = LabelEncoder()
            train[col] = train[col].map(str)
            train[col] = encoder.fit_transform(train[col])
            encoders[col] = encoder
            text_temp.append(col)
    year_encoder[year] = encoders
    int_col[year] = int_temp
    text_col[year] = text_temp

    feature = train.columns.drop(['idx', 'knowcode'])

    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train[feature], train['knowcode'])

    for col in test.columns:
      try:
          test[col] = test[col].map(int)
      except:
          encoder = year_encoder[year][col]
          test[col] = test[col].map(str)
          category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
          test[col] = test[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
    feature = test.columns.drop(['idx'])
    pred = model.predict(test[feature])
    rf_predicts.extend(pred)

2017
2018
2019
2020


In [90]:
int_col['2017']

['idx',
 'aq1_1',
 'aq1_2',
 'aq2_1',
 'aq2_2',
 'aq3_1',
 'aq3_2',
 'aq4_1',
 'aq4_2',
 'aq5_1',
 'aq5_2',
 'aq6_1',
 'aq6_2',
 'aq7_1',
 'aq7_2',
 'aq8_1',
 'aq8_2',
 'aq9_1',
 'aq9_2',
 'aq10_1',
 'aq10_2',
 'aq11_1',
 'aq11_2',
 'aq12_1',
 'aq12_2',
 'aq13_1',
 'aq13_2',
 'aq14_1',
 'aq14_2',
 'aq15_1',
 'aq15_2',
 'aq16_1',
 'aq16_2',
 'aq17_1',
 'aq17_2',
 'aq18_1',
 'aq18_2',
 'aq19_1',
 'aq19_2',
 'aq20_1',
 'aq20_2',
 'aq21_1',
 'aq21_2',
 'aq22_1',
 'aq22_2',
 'aq23_1',
 'aq23_2',
 'aq24_1',
 'aq24_2',
 'aq25_1',
 'aq25_2',
 'aq26_1',
 'aq26_2',
 'aq27_1',
 'aq27_2',
 'aq28_1',
 'aq28_2',
 'aq29_1',
 'aq29_2',
 'aq30_1',
 'aq30_2',
 'aq31_1',
 'aq31_2',
 'aq32_1',
 'aq32_2',
 'aq33_1',
 'aq33_2',
 'aq34_1',
 'aq34_2',
 'aq35_1',
 'aq35_2',
 'aq36_1',
 'aq36_2',
 'aq37_1',
 'aq37_2',
 'aq38_1',
 'aq38_2',
 'aq39_1',
 'aq39_2',
 'aq40_1',
 'aq40_2',
 'aq41_1',
 'aq41_2',
 'bq1',
 'bq2',
 'bq3',
 'bq4',
 'bq5',
 'bq5_1',
 'bq6',
 'bq7',
 'bq8_1',
 'bq8_2',
 'bq8_3',
 'bq9',
 'bq

In [89]:
text_col['2017']

['bq4_1a',
 'bq4_1b',
 'bq4_1c',
 'bq5_2',
 'bq19_1',
 'bq30',
 'bq31',
 'bq32',
 'bq33',
 'bq34',
 'bq38_1']

In [77]:
submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv') # sample submission 불러오기

In [78]:
submission['knowcode'] = rf_predicts

In [79]:
submission

,idx,knowcode
0,0,29401
1,1,151108
2,2,9999999
3,3,121102
4,4,412003
...,...,...
35226,35244,851101
35227,35245,411103
35228,35246,833001
35229,35247,140701


In [80]:
submission.to_csv('/content/gdrive/MyDrive/know/submission.csv', index=False)